In [2]:
import cv2
import numpy as np
import os
from tqdm import tqdm

In [9]:
def undistort_video(input_path, output_path, calibration_images=None):
    """
    Remove lens distortion from a video file
    
    Parameters:
    - input_path: Path to the input video file
    - output_path: Path to save the undistorted video
    - calibration_images: Optional list of paths to chessboard images for calibration
                         If None, uses default estimation values
    """
    # Use estimated values for a typical wide-angle camera
    # These are default values, adjust based on your specific camera if known
    print("Using default camera parameters (no calibration images provided)")
    # Camera matrix (focal length and optical centers)
    mtx = np.array([
        [1000, 0, 960],  # fx, 0, cx
        [0, 1000, 540],  # 0, fy, cy
        [0, 0, 1]
    ])
    # Distortion coefficients [k1, k2, p1, p2, k3]
    dist = np.array([[-0.3, 0.1, 0, 0, -0.02]])

    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Couldn't open video file {input_path}")
        return False
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate optimal camera matrix
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (width, height), 1, (width, height))
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"Processing video with {frame_count} frames...")
    frame_idx = 0
    
    # Process each frame
    with tqdm(total=frame_count) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Undistort the frame
            dst = cv2.undistort(frame, mtx, dist, None, newcameramtx)
            
            # Crop the image (optional)
            x, y, w, h = roi
            dst = dst[y:y+h, x:x+w]
            
            # Resize back to original dimensions if needed
            if dst.shape[1] != width or dst.shape[0] != height:
                dst = cv2.resize(dst, (width, height))
            
            # Write the undistorted frame
            out.write(dst)
            
            frame_idx += 1
            pbar.update(1)
    
    # Release resources
    cap.release()
    out.release()
    
    print(f"Undistorted video saved to {output_path}")
    return True

In [10]:
undistort_video("10secs.mp4", "undist.mp4")

Using default camera parameters (no calibration images provided)
Processing video with 301 frames...


100%|████████████████████████████████████████▊| 300/301 [00:04<00:00, 60.26it/s]

Undistorted video saved to undist.mp4


True

In [11]:
def detect_court_lines(frame):
    """
    Detect court lines using Hough Line Transform
    Returns the court corners if found
    """
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Use adaptive thresholding to identify court lines
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Find edges
    edges = cv2.Canny(thresh, 50, 150)
    
    # Dilate to connect edge components
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Find lines using HoughLinesP
    lines = cv2.HoughLinesP(
        dilated, 1, np.pi/180, threshold=50, 
        minLineLength=50, maxLineGap=10
    )
    
    if lines is None:
        return None
    
    # Draw lines on a copy of the frame for visualization
    line_frame = frame.copy()
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # For debugging: save the line detection result
    cv2.imwrite("court_lines_detected.jpg", line_frame)
    
    # Group lines into horizontal and vertical
    h_lines = []
    v_lines = []
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Calculate angle
        angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
        
        # Group based on angle
        if angle < 45 or angle > 135:  # Horizontal lines
            h_lines.append(line[0])
        else:  # Vertical lines
            v_lines.append(line[0])
    
    # If we don't have enough lines, return None
    if len(h_lines) < 2 or len(v_lines) < 2:
        return None
    
    # Find boundary lines (furthest lines in each direction)
    h_lines = sorted(h_lines, key=lambda line: (line[1] + line[3]) / 2)  # Sort by y-coordinate
    v_lines = sorted(v_lines, key=lambda line: (line[0] + line[2]) / 2)  # Sort by x-coordinate
    
    top_line = h_lines[0]
    bottom_line = h_lines[-1]
    left_line = v_lines[0]
    right_line = v_lines[-1]
    
    # Find intersections of these lines to get court corners
    top_left = line_intersection(top_line, left_line)
    top_right = line_intersection(top_line, right_line)
    bottom_left = line_intersection(bottom_line, left_line)
    bottom_right = line_intersection(bottom_line, right_line)
    
    # Convert to integer points
    corners = [
        (int(top_left[0]), int(top_left[1])),
        (int(top_right[0]), int(top_right[1])),
        (int(bottom_right[0]), int(bottom_right[1])),
        (int(bottom_left[0]), int(bottom_left[1]))
    ]
    
    return corners

In [13]:
def line_intersection(line1, line2):
    """Find the intersection point of two lines"""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2
    
    # Line 1 represented as a1x + b1y = c1
    a1 = y2 - y1
    b1 = x1 - x2
    c1 = a1 * x1 + b1 * y1
    
    # Line 2 represented as a2x + b2y = c2
    a2 = y4 - y3
    b2 = x3 - x4
    c2 = a2 * x3 + b2 * y3
    
    determinant = a1 * b2 - a2 * b1
    
    if determinant == 0:
        # Lines are parallel
        return (0, 0)
    else:
        x = (b2 * c1 - b1 * c2) / determinant
        y = (a1 * c2 - a2 * c1) / determinant
        return (x, y)


In [14]:
def manual_court_selection(frame):
    """
    Allow manual selection of court corners if automatic detection fails
    """
    corners = []
    
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            corners.append((x, y))
            # Draw circle at selected point
            cv2.circle(display_frame, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow('Select Court Corners', display_frame)
            
            if len(corners) == 4:
                cv2.destroyWindow('Select Court Corners')
    
    # Create a copy of the frame for display
    display_frame = frame.copy()
    cv2.namedWindow('Select Court Corners')
    cv2.setMouseCallback('Select Court Corners', mouse_callback)
    
    # Display instructions
    text = "Click on 4 court corners in order: Top-Left, Top-Right, Bottom-Right, Bottom-Left"
    y_pos = 30
    for i, line in enumerate(text.split(', ')):
        cv2.putText(display_frame, line, (10, y_pos + i*30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow('Select Court Corners', display_frame)
    cv2.waitKey(0)
    
    return corners if len(corners) == 4 else None

In [15]:
def apply_perspective_transform(frame, corners, target_width=800, target_height=400):
    """
    Apply perspective transformation to get a top-down view of the court
    """
    # Define the destination points (rectangle)
    dst_points = np.array([
        [0, 0],                      # Top-left
        [target_width, 0],           # Top-right
        [target_width, target_height], # Bottom-right
        [0, target_height]           # Bottom-left
    ], dtype=np.float32)
    
    # Convert corners to numpy array
    src_points = np.array(corners, dtype=np.float32)
    
    # Compute perspective transform matrix
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Apply transformation
    warped = cv2.warpPerspective(frame, M, (target_width, target_height))
    
    return warped, M

In [16]:
def process_video_with_perspective_transform(input_path, output_path):
    """
    Process a video to detect court and transform perspective
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Couldn't open video file {input_path}")
        return False
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Read first frame for court detection
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read the first frame")
        return False
    
    # Try automatic court detection
    corners = detect_court_lines(first_frame)
    
    # If automatic detection fails, use manual selection
    if corners is None or len(corners) != 4:
        print("Automatic court detection failed. Please select court corners manually.")
        corners = manual_court_selection(first_frame)
        
        if corners is None or len(corners) != 4:
            print("Court corner selection failed")
            return False
    
    # Apply perspective transform
    target_width, target_height = 800, 400  # Standard padel court dimensions (scaled)
    _, perspective_matrix = apply_perspective_transform(first_frame, corners, target_width, target_height)
    
    # Create video writer for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))
    
    # Reset video capture to beginning
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    print(f"Processing video with {frame_count} frames...")
    
    # Process each frame
    with tqdm(total=frame_count) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Apply perspective transform
            warped_frame, _ = apply_perspective_transform(
                frame, corners, target_width, target_height
            )
            
            # Write transformed frame
            out.write(warped_frame)
            
            pbar.update(1)
    
    # Release resources
    cap.release()
    out.release()
    
    print(f"Perspective-corrected video saved to {output_path}")
    
    # Save the perspective matrix for future use in player tracking
    np.save("perspective_matrix.npy", perspective_matrix)
    print("Perspective transformation matrix saved to perspective_matrix.npy")
    
    return True, perspective_matrix

In [17]:
succ, trans = process_video_with_perspective_transform("undist.mp4", "new.mp4")

Processing video with 300 frames...


100%|████████████████████████████████████████| 300/300 [00:00<00:00, 353.48it/s]

Perspective-corrected video saved to new.mp4
Perspective transformation matrix saved to perspective_matrix.npy


In [ ]:
def process_videos_in_directory(input_dir, output_dir, calibration_dir=None):
    """
    Process all .mp4 videos in a directory
    
    Parameters:
    - input_dir: Directory containing input videos 
    - output_dir: Directory to save processed videos
    - calibration_dir: Optional directory with calibration images
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get list of calibration images if provided
    calibration_images = None
    if calibration_dir and os.path.exists(calibration_dir):
        calibration_images = [os.path.join(calibration_dir, f) for f in os.listdir(calibration_dir)
                             if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Process each video
    for filename in os.listdir(input_dir):
        if filename.endswith('.mp4'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"undistorted_{filename}")
            print(f"Processing {filename}...")
            undistort_video(input_path, output_path, calibration_images)

if __name__ == "__main__":
    # Example usage
    input_video = "paddle_game.mp4"
    output_video = "undistorted_paddle_game.mp4"
    
    # Option 1: Process a single video
    undistort_video(input_video, output_video)
    
    # Option 2: Process all videos in a directory
    # process_videos_in_directory("input_videos", "output_videos", "calibration_images")